In [26]:
from learners.s_learner import S_learner
import tensorflow as tf 
from tensorflow import keras
import numpy as np
from dataloader import load_IHDP_data
from visualizations import plot_cates

In [3]:
import pandas as pd

In [4]:
jobs = pd.read_csv('jobsdata.csv')

In [7]:
jobs
# treatment column is called 'treat' 
# an outcome of interest is 'job_seek', which measures the level of job-search self-efficacy
#    with values from 1 to 5
# some standard covariates are sex, age, marital, nonwhite, educ, and income

,Unnamed: 0,treat,econ_hard,depress1,sex,age,occp,marital,nonwhite,educ,income,job_seek,depress2,work1,comply,control,job_dich,job_disc
0,1,1,3.00,1.91,1,34.167122,professionals,married,non.white1,gradwk,50k+,4.833333,1.727273,psyemp,0,treat,1,4
1,2,1,3.67,1.36,0,26.101370,operatives/kindred wrks,nevmarr,white0,somcol,15t24k,3.833333,2.000000,psyemp,0,treat,0,3
2,3,1,4.00,2.09,1,35.021919,operatives/kindred wrks,nevmarr,non.white1,somcol,25t39k,4.500000,2.181818,psyump,0,treat,1,4
3,4,0,2.33,1.45,0,27.487671,manegerial,married,white0,bach,25t39k,3.666667,1.545455,psyump,0,control,0,3
4,5,1,1.33,1.73,1,31.610958,clerical/kindred,separtd,non.white1,highsc,25t39k,2.500000,2.363636,psyump,1,treat,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,895,1,3.00,1.36,0,38.205479,craftsmen/foremen/kindred,married,white0,highsc,25t39k,4.166667,1.727273,psyump,0,treat,1,3
895,896,1,4.00,2.55,0,40.972603,professionals,married,white0,somcol,25t39k,3.000000,3.636364,psyump,1,treat,0,2
896,897,0,5.00,1.00,1,48.909588,professionals,married,non.white1,gradwk,50k+,5.000000,1.454545,psyump,0,control,1,4
897,898,0,4.33,1.91,1,25.043835,clerical/kindred,married,white0,somcol,lt15k,4.000000,1.545455,psyemp,0,control,1,3


In [29]:
xt = jobs[['treat', 'sex', 'age', 'depress1', 'econ_hard']]


S-learner

In [30]:
loss_fn = tf.keras.losses.MeanSquaredError() #specify the loss

val_split=0.2
batch_size=64
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)

sgd_callbacks = [
        keras.callbacks.TerminateOnNaN(),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, min_delta=0.), 
        #40 is Shi's recommendation for this dataset, but you should tune for your data 
        keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=0., cooldown=0, min_lr=0),
    ]
#optimzier hyperparameters
sgd_lr = 1e-5
momentum = 0.9

In [66]:
sgd_lr_test = [1e-1, 1e-3, 1e-5, 1e-7, 1e-9]
momentum_test = [0.75, 0.9]
dims_test = [100, 200, 400, 500, 900]

losses = []

for lr in sgd_lr_test:
    #for momentum in momentum_test:
    for dim in dims_test:
                    
        s_learner = S_learner(dim)

        loss_fn = tf.keras.losses.MeanSquaredError() #specify the loss

        s_learner.compile(optimizer= keras.optimizers.Adam(learning_rate=lr),#, nesterov=True),
                loss=loss_fn,
                metrics=loss_fn)

        #'ys' transformed y
        s_learner.fit(x=xt,y=jobs['job_seek'],
                        validation_split=.2,
                        epochs=300,
                        batch_size=64,
                        callbacks=sgd_callbacks,
                        verbose=0)

        loss = s_learner.loss(jobs['job_seek'], s_learner.predict(xt)).numpy()

        losses.append([lr, dim, loss])
        print("Done")


Epoch 20: ReduceLROnPlateau reducing learning rate to 0.05000000074505806.

Epoch 28: ReduceLROnPlateau reducing learning rate to 0.02500000037252903.

Epoch 33: ReduceLROnPlateau reducing learning rate to 0.012500000186264515.

Epoch 38: ReduceLROnPlateau reducing learning rate to 0.0062500000931322575.

Epoch 43: ReduceLROnPlateau reducing learning rate to 0.0031250000465661287.

Epoch 48: ReduceLROnPlateau reducing learning rate to 0.0015625000232830644.

Epoch 53: ReduceLROnPlateau reducing learning rate to 0.0007812500116415322.

Epoch 58: ReduceLROnPlateau reducing learning rate to 0.0003906250058207661.

Epoch 63: ReduceLROnPlateau reducing learning rate to 0.00019531250291038305.

Epoch 68: ReduceLROnPlateau reducing learning rate to 9.765625145519152e-05.

Epoch 73: ReduceLROnPlateau reducing learning rate to 4.882812572759576e-05.

Epoch 78: ReduceLROnPlateau reducing learning rate to 2.441406286379788e-05.
29/29 [==============================] - 0s 3ms/step
Done

Epoch 24:

In [60]:
losses # for SGD

[[0.1, 0.75, 100, nan],
 [0.1, 0.75, 200, nan],
 [0.1, 0.9, 100, nan],
 [0.1, 0.9, 200, nan],
 [0.001, 0.75, 100, 0.5393616],
 [0.001, 0.75, 200, nan],
 [0.001, 0.9, 100, nan],
 [0.001, 0.9, 200, nan],
 [1e-05, 0.75, 100, 0.54831076],
 [1e-05, 0.75, 200, 0.54383457],
 [1e-05, 0.9, 100, 0.54781026],
 [1e-05, 0.9, 200, 0.5494427],
 [1e-07, 0.75, 100, 0.7556977],
 [1e-07, 0.75, 200, 1.1327391],
 [1e-07, 0.9, 100, 1.1106572],
 [1e-07, 0.9, 200, 0.8156351],
 [1e-09, 0.75, 100, 56.58402],
 [1e-09, 0.75, 200, 11.450971],
 [1e-09, 0.9, 100, 6.7916665],
 [1e-09, 0.9, 200, 5.134691]]

In [67]:
losses # for Adam

[[0.1, 100, 0.5300556],
 [0.1, 200, 1.1959561],
 [0.1, 400, 0.5300973],
 [0.1, 500, 0.5300699],
 [0.1, 900, 13.299489],
 [0.001, 100, 0.5620591],
 [0.001, 200, 0.5618848],
 [0.001, 400, 0.5567285],
 [0.001, 500, 0.5415936],
 [0.001, 900, 0.53746504],
 [1e-05, 100, 0.54704165],
 [1e-05, 200, 0.5496462],
 [1e-05, 400, 0.544023],
 [1e-05, 500, 0.54978573],
 [1e-05, 900, 0.55109537],
 [1e-07, 100, 1.2805567],
 [1e-07, 200, 3.696349],
 [1e-07, 400, 0.80296296],
 [1e-07, 500, 0.6333041],
 [1e-07, 900, 0.6464903],
 [1e-09, 100, 11.147198],
 [1e-09, 200, 14.985779],
 [1e-09, 400, 22.19937],
 [1e-09, 500, 13.275995],
 [1e-09, 900, 27.632545]]

In [33]:
x = jobs[['sex', 'age', 'depress1', 'econ_hard']]

In [68]:
#create fake ones and zeros to feed network
zeros=np.expand_dims(np.zeros(x.shape[0]),1)
ones=np.expand_dims(np.ones(x.shape[0]),1)
x_untreated = np.concatenate([x, zeros], 1)
x_treated = np.concatenate([x, ones], 1)
y0_pred_slearner=s_learner.predict(x_untreated)
y1_pred_slearner=s_learner.predict(x_treated)

29/29 [==============================] - 0s 14ms/step


In [50]:
s_learner.loss(jobs['job_seek'], s_learner.predict(xt)).numpy()

29/29 [==============================] - 0s 4ms/step


0.546997

In [69]:
tf.reduce_mean((y1_pred_slearner-y0_pred_slearner).squeeze())

<tf.Tensor: shape=(), dtype=float32, numpy=-0.023824032>

Linear Regression

In [72]:
import sklearn

In [74]:
from sklearn.linear_model import LinearRegression

In [75]:
model = LinearRegression()

In [76]:
model.fit(xt, jobs['job_seek'])

LinearRegression()

In [77]:
model.coef_

array([ 0.06012222,  0.02406536,  0.00413283, -0.26331541,  0.10260872])

In [78]:
xt

,treat,sex,age,depress1,econ_hard
0,1,1,34.167122,1.91,3.00
1,1,0,26.101370,1.36,3.67
2,1,1,35.021919,2.09,4.00
3,0,0,27.487671,1.45,2.33
4,1,1,31.610958,1.73,1.33
...,...,...,...,...,...
894,1,0,38.205479,1.36,3.00
895,1,0,40.972603,2.55,4.00
896,0,1,48.909588,1.00,5.00
897,0,1,25.043835,1.91,4.33
